<a href="https://colab.research.google.com/github/yifan-zhou19/Policy_migration/blob/master/Access_Polich_Migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#define ABAC policy (university samples)
U=[{'uid':'applicant1','position':'applicant'},
 {'uid':'csStu1','position':'student','udepartment':'cs','crsTaken':['cs101']},
 {'uid':'csStu2','position':'student','udepartment':'cs','crsTaken':['cs101','cs601']},
 {'uid':'csFac1','position':'faculty','udepartment':'cs','crsTaken':['cs101']},
 {'uid':'csFac2','position':'faculty','udepartment':'cs','crsTaken':['cs601']},
 {'uid':'csChair','isChair':True,'udepartment':'cs'},
 {'uid':'eeFac1','position':'faculty','udepartment':'ee','crsTaken':['ee101']},
 {'uid':'eeFac2','position':'faculty','udepartment':'ee','crsTaken':['ee601']},
 {'uid':'eeChair','isChair':True,'udepartment':'ee'},
 {'uid':'registrar1','position':'staff','udepartment':'registrar'},
 {'uid':'admissions1','position':'staff','udepartment':'admissions'},
]
R=[{'rid':'application1','type':'application','student':'applicant1'},
 {'rid':'csStu1application','type':'application','student':'csStu1'},
 {'rid':'csStu2application','type':'application','student':'csStu2'},
 {'rid':'eeStu1application','type':'application','student':'eeStu1'},
 {'rid':'eeStu2application','type':'application','student':'eeStu2'},
 {'rid':'cs101gradebook','type':'gradebook','rdepartment':'cs','crs':'cs101'},
 {'rid':'cs601gradebook','type':'gradebook','rdepartment':'cs','crs':'cs601'},
 {'rid':'ee101gradebook','type':'gradebook','rdepartment':'ee','crs':'ee101'},
 {'rid':'ee601gradebook','type':'gradebook','rdepartment':'ee','crs':'ee601'},
 {'rid':'cs101roster','type':'roster','rdepartment':'cs','crs':'cs101'},
 {'rid':'cs601roster','type':'roster','rdepartment':'cs','crs':'cs601'},
 {'rid':'ee101roster','type':'roster','rdepartment':'ee','crs':'ee101'},
 {'rid':'ee601roster','type':'roster','rdepartment':'ee','crs':'ee601'},
 {'rid':'csStu1trans','type':'transcript','rdepartment':'cs','student':'csStu1'},
 {'rid':'csStu2trans','type':'transcript','rdepartment':'cs','student':'csStu2'},
 {'rid':'eeStu1trans','type':'transcript','rdepartment':'ee','student':'eeStu1'},
 {'rid':'eeStu2trans','type':'transcript','rdepartment':'ee','student':'eeStu2'},

]
UA=['position','udepartment','crsTaken','crsTaught','isChair','uid']
RA=['type','student','rdepartment', 'crs','rid']
O=['readMyScores','addScore', 'readScore','changeScore', 'assignGrade','read','write','checkStatus','setStatus']
RU=[{# a user can read his/her own scores in gradebooks for courses he/she has taken. 
  'expressions':[{'eu':{ },'er':{'type':('is','gradebook'),'crs':('in','crsTaken')}}],'operations':['readMyScores'],'decision':'allow'}, 
# a user (the instructor or TA) can add scores and read scores in the gradebook for courses he/she is teaching. 
 {'expressions':[{'eu':{ },'er':{'type':('is','gradebook'),'crs':('in','crsTaught')}}],'operations':['addScore', 'readScore'],'decision':'allow'}, 
 # the instructor for a course (i.e., a faculty teaching the course) can change scores and assign grades in the gradebook for that course. 
{'expressions':[{'eu':{'position':('is','faculty')},'er':{'type':('is','gradebook'),'crs':('in','crsTaught')}}],'operations':['changeScore','assignGrade'],'decision':'allow'}, 
 # a user in the registrar's office can read and modify all rosters.
{'expressions':[{'eu':{'department':('is','registrar')},'er':{'type':('is','roster')}}],'operations':['read','write'],'decision':'allow'}, 
# the instructor for a course (i.e., a faculty teaching the course) can read the course roster
{'expressions':[{'eu':{'position':('is','faculty')},'er':{'type':('is','roster'),'crs':('in','crsTaught')}}],'operations':['read'],'decision':'allow'}, 
# a user can read his/her own transcript.
{'expressions':[{'eu':{ },'er':{'type':('is','transcript'),'student':('=','uid')}}],'operations':['read'],'decision':'allow'}, 
# the chair of a department can read the transcripts of all students in that department.
{'expressions':[{'eu':{'icChair':('is',True)},'er':{'type':('is','transcript'),'rdepartment':('is','udepartment')}}],'operations':['read'],'decision':'allow'}, 
# a user in the registrar's office can read every student's transcript.
{'expressions':[{'eu':{'udepartment':('is','registrar')},'er':{'type':('is','transcript')}}],'operations':['read'],'decision':'allow'}, 
# a user can check the status of his/her own application 
{'expressions':[{'eu':{ },'er':{'type':('is','application'),'student':('=','uid')}}],'operations':['checkStatus'],'decision':'allow'}, 
# a user in the admissions office can read, and update the status of, every application. 
{'expressions':[{'eu':{'udepartment':('is','admission')},'er':{'type':('is','application')}}],'operations':['read','setStatus'],'decision':'allow'}
]
def dU(user,attribute):
  if attribute in user.keys():
    return user[attribute]

def dR(resource,attribute):
  if attribute in resource.keys():
    return resource[attribute]

ABAC_university=(U,R,O,UA,RA,dU,dR,RU)


In [ ]:

def satisfied(u,r,expressions):
  for expression in expressions:
    eu=expression['eu']
    er=expression['er']
    #for all the keys in eu get the constrina of this key
    #if u does not satisfy this const return false
    for key in eu.keys():
      op,c=eu[key]
      if op == 'is':
        if dU(u,key) != c:
          return False

    #for all the keys in er get the constrinat of the key
    for key in er.keys():
      op,c=er[key]
      if op =='is':
        if dR(r,key)!=c:
        #  print(dR(r,key),c)
          return False
      elif op == '=':
        if dR(r,key)!=dU(u,c) and dR(r,key)!=dR(r,c):
          return False
      elif op == 'in':
        if dR(r,key) not in dU(u,c) and dR(r,key) not in dR(r,c):
          return False
  
  return True


def check_ABAC_policy(request,abac_policy):
  u,r,o =request
  _,_,_,_,_,_,_,rules = abac_policy
  for rule in rules:
    if o in rule['operations']:
      if satisfied(u,r,rule['expressions']):
        return True
  return False
check_ABAC_policy((U[1],R[5],'readMyScores'),ABAC_university)

True